This is a messy file that was used iteratively to make [Extended-Full]LatLong-tweet-test.csv and all of the day csv's. Refer to the comments to see what each part really does.

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point,Polygon
import json

In [ ]:
df_columbia = gpd.read_file('../municipal_boundaries/MGN2012_MUNICIPIO_URBANCLUSTERS.shp')
df_tweets = pd.read_csv('../data/fake_tweets.csv')
df_centroids = pd.read_csv('./region-to-centroid.csv')

In [ ]:
# extracts the region (index of the shapefile) from a latitude and longitude
def long_lat_to_region(df, long, lat):
    bp = Point(long, lat)
    for index, row in df.iterrows():
        if bp.within(row['geometry']):
            return index

In [ ]:
df_tweets.head()

In [ ]:
# a hashmap that remembers a region's name to its index. It is used as to memoize the lookup process.
location_to_index = None
with open('region_to_region_index.json', 'r') as fp:
    location_to_index = json.load(fp)

In [ ]:
# this takes the fake tweets and converts the latitude and longitude's into region numbers and each region's centroid
# this will make a comprehensive dataset called [Extended-Full]LatLong-tweet-test.csv that will be used during visualization
toAddOldRegion = []
toAddNewRegion = []

toAddOldLong = []
toAddOldLat = []

toAddNewLong = []
toAddNewLat = []

for i in range(len(df_tweets)):
    if (i % 1000 == 0):
        print(i)
    data = df_tweets.iloc[i]

    old_region = long_lat_to_region(df_columbia, data.Longitude, data.Latitude)
    new_region = long_lat_to_region(df_columbia, data.New_Long, data.New_Lat)
    
    old_centroid = df_centroids[df_centroids['Name'] == old_region]
    new_centroid = df_centroids[df_centroids['Name'] == new_region]
    
    if (len(old_centroid) == 0 or len(new_centroid) == 0):
        toAddOldRegion.append(None)
        continuefr
    
    location_to_index[data.Location] = old_region
    location_to_index[data.New_Loc] = new_region
    
    toAddOldRegion.append(old_region)
    toAddNewRegion.append(new_region)
    
    toAddOldLong.append(old_centroid.Longitude.values[0])
    toAddOldLat.append(old_centroid.Latitude.values[0])
    
    toAddNewLong.append(new_centroid.Longitude.values[0])
    toAddNewLat.append(new_centroid.Latitude.values[0])

In [ ]:
df_tweets['Old Region'] = toAddOldRegion
# drops rows where match wasn't found
df_tweets.dropna(subset=['Old Region'], inplace=True)
df_tweets['New Region'] = toAddNewRegion

df_tweets['Old_Region_Long'] = toAddOldLong
df_tweets['Old_Region_Lat'] = toAddOldLat

df_tweets['New_Region_Long'] = toAddNewLong
df_tweets['New_Region_Lat'] = toAddNewLat

In [ ]:
# the new csv
\df_tweets.head()

In [ ]:
# gets how many people went from one region to another
df_dist = df_tweets.groupby(['Old Region', 'New Region']).count().reset_index().nlargest(20, 'UserID')

In [ ]:
# simplifies the dataframe to show only what's important
df_clean = pd.DataFrame.from_dict({'Old Region': df_dist['Old Region'], 'New Region': df_dist['New Region'], 'Number': df_dist['UserID']})

In [ ]:
# this creates a csv of the top 20 regional movements
df_clean.to_csv('../data/movement.csv')

In [ ]:
df_dist_full = df_tweets.groupby(['Old Region', 'New Region']).count().reset_index()

In [ ]:
df_clean_full = pd.DataFrame.from_dict({'Old Region': df_dist_full['Old Region'], 'New Region': df_dist_full['New Region'], 'Number': df_dist_full['UserID']})

In [ ]:
# this creates a csv of all regional movements
df_clean_full.to_csv('../data/movement_full.csv')

In [ ]:
# this was done AFTER. this reads the csv's created. Note they were moved by me to the same directory, although the code saved them elsewhere.
df_move = pd.read_csv('movement.csv')
df_loc = pd.read_csv('region-to-centroid.csv')

In [ ]:
# uses df_loc to turn a region number into the centroid
def apply_long_find(region):
    return df_loc[df_loc['Name'] == region].Longitude.values[0]

def apply_lat_find(region):
    return df_loc[df_loc['Name'] == region].Latitude.values[0]

In [ ]:
df_move['Old Longitude'] = df_move['Old Region'].apply(apply_long_find)
df_move['Old Latitude'] = df_move['Old Region'].apply(apply_lat_find)
df_move['New Longitude'] = df_move['New Region'].apply(apply_long_find)
df_move['New Latitude'] = df_move['New Region'].apply(apply_lat_find)

In [ ]:
# refines movement.csv to be the centroid's of each of the regions
df_move.to_csv('movement.csv')

In [ ]:
df = pd.read_csv('../data/LatLong-tweet-test.csv')

In [ ]:
df_tweets.dropna(subset=['created_at'], inplace=True)

In [ ]:
# created the Year, Month, Day, Hour, Minute, and Second columns of [Extended-Full]LatLong-tweet-test.csv and the day csvs.
def parse_date_col(df, datecol):
    years = []
    months = []
    days = []
    hours = []
    minutes = []
    seconds = []
    for index, row in df.iterrows():
        large, small = row[datecol].split(' ')
        year, month, day = large.split('-')
        hour, minute, second = small.split(':')
        years.append(year)
        months.append(month)
        days.append(day)
        hours.append(hour)
        minutes.append(minute)
        seconds.append(second)
    df['Year'] = years
    df['Month'] = months
    df['Day'] = days
    df['Hour'] = hours
    df['Minute'] = minutes
    df['Second'] = seconds

In [ ]:
df.head()

In [ ]:
parse_date_col(df_tweets, 'created_at')

In [ ]:
df_tweets.to_csv('../data/[Extended-Full]LatLong-tweet-test.csv')

In [ ]:
# reopens the hashmap for further use
location_to_index = None
with open('region_to_region_index.json', 'r') as fp:
    location_to_index = json.load(fp)
  
# function that checks the hashmap and if it's not in the hashmap, it updates the hashmap (memoization) after doing the expensive lookup process
def region_to_region_index(region, long, lat):
    if region not in location_to_index:
        location_to_index[region] = long_lat_to_region(df_columbia, long, lat)
    return location_to_index[region]

In [ ]:
df = pd.read_csv('../data/[Extended]LatLong-tweet-test.csv')

In [ ]:
df.dropna(subset=['Longitude', 'Latitude'], inplace=True)

In [ ]:
toAdd = []
for index, row in df.iterrows():
    try:
        toAdd.append(region_to_region_index(row['Location'], float(row['Longitude']), float(row['Latitude'])))
    except:
        toAdd.append(None)

In [ ]:
df['Region_Number'] = toAdd
df.dropna(subset=['Region_Number'], inplace=True)

In [ ]:
# creates the day csvs!
for name, group in df_tweets.groupby(['Day']):
    file = '[Day ' + str(name) + ']-tweets.csv'
    group.sort_values(by=['Hour'], inplace=True)
    group.to_csv('./' + file)

In [ ]:
# saves hashmap
with open('region_to_region_index.json', 'w') as fp:
    json.dump(location_to_index, fp)